<a href="https://colab.research.google.com/github/bdilpreet66/CNN/blob/main/COCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2023-04-28 03:25:56--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.205, 3.5.9.11, 54.231.169.89, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.205|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip        96%[==================> ]  17.44G  77.7MB/s    eta 8s     

In [ ]:
!unzip train2017.zip -d train2017
!unzip annotations_trainval2017.zip -d annotations

In [ ]:
from pycocotools.coco import COCO

# Initialize the COCO API for instance annotations
coco = COCO('annotations/instances_train2017.json')

# Load the categories in the dataset
categories = coco.loadCats(coco.getCatIds())

# Load the annotations for the dataset
annotations = coco.loadAnns(coco.getAnnIds())